In [3]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch
from six.moves import urllib
DEVICE=torch.device('cuda' if torch.cuda.is_available else 'cpu')
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transforms.Compose([transforms.Resize((32, 32)),transforms.ToTensor()]))
test_data = datasets.MNIST(root='data', train=False,
                                  download=False, transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor()]))

100.1%Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw
113.5%Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw
100.4%Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw
180.4%C:\Users\3neut\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw
Processing...
Done!


In [ ]:
data_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=60000,
                                          shuffle=True,
                                        )
print(data_loader.dataset.train_data.size())
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=(5,5)) # 5x5+1 params
        self.subsampling=nn.MaxPool2d(kernel_size=(2,2),stride=2)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=(5,5)) # 5x5+1 params
        self.conv3=nn.Conv2d(in_channels=16,out_channels=120,kernel_size=(5,5)) # 5x5+1 params
        self.fc1=nn.Linear(120,84)
        self.fc2=nn.Linear(84,10)
        self.log_softmax=nn.LogSoftmax(dim=-1)
    
    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.subsampling(x)
        x=F.relu(self.conv2(x))
        x=self.subsampling(x)
        x=F.relu(self.conv3(x))
        x=x.view(-1,120)
        x=F.relu(self.fc1(x))
        x=self.log_softmax(x)
        return x

net = LeNet5().to(DEVICE)
optimizer = optim.Adam(net.parameters(), lr=2e-3)
parameter_list=list()
def train(log_interval, model, device, train_loader, optimizer, epoch): # model제작이 첫번째
    model.train() #train모드로 설정
    running_loss =0.0
    criterion = nn.CrossEntropyLoss() #defalut is mean of mini-batchsamples, loss type설정
    # loss함수에 softmax 함수가 포함되어있음
    for batch_idx, (data, target) in enumerate(train_loader): # 몇개씩(batch size) 로더에서 가져올지 정함 #enumerate로 batch_idx표현
        data, target = data.to(device), target.to(device) #gpu로 올림
        optimizer.zero_grad()# optimizer zero로 초기화
        output = model(data) #model에서 입력과 출력이 나옴 batch 수만큼 들어가서 batch수만큼 결과가 나옴 (1개 인풋 1개 아웃풋 아님)
        loss = criterion(output, target) #결과와 target을 비교하여 계산 

        loss.backward() #역전파
        optimizer.step() # step
        p_groups=optimizer.param_groups # group에 각 layer별 파라미터
        parameter_list.append([])
        for p in p_groups:
            for p_layers in p['params']:
                parameter_list[-1].append(p_layers.view(-1).clone())
        
        running_loss += loss.item()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), running_loss/log_interval))


In [ ]:
epochs=50
for epoch in range(1, epochs + 1):
    train(100,net,DEVICE,data_loader,optimizer,epoch)

'''
 자료구조
parameter_list
1dim: time
2dim: layer

저장시
x 축 내용: parameters의 grad
x 축 : time
y 축 : param
'''
import time
import csv
param_size=list()
params_write=list()
f=open('grad.csv',mode='w')
fwriter=csv.writer(f)
a=time.time()
for t,params in enumerate(parameter_list):
    if t==1:
        for i,p in enumerate(params):# 각 layer의 params
            param_size.append(p.size())
    params_write=torch.cat(params,dim=0).tolist()
    fwriter.writerow(params_write)
    if t % 10 == 0:
        print("\r step {} done".format(t),end='')
f.close()
b=time.time()
print(b-a,"s")
print(param_size)